In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import numpy as np

import torch.nn.functional as nnf
import sys
from typing import Tuple, List, Union, Optional

import clip
import os
from tqdm import tqdm, trange

from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup

import skimage.io as io
import PIL.Image
from IPython.display import Image 
from enum import Enum

In [ ]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

In [ ]:
#Data loaders


In [1]:

# Hyperparameters
d_model = 512  # Dimension of the model
output_dim = 32  # Output dimension of the MLP
num_layers = 6  # Number of transformer layers
num_heads = 8  # Number of heads in multi-headed attention
dim_feedforward = 2048  # Dimension of the feedforward network
dropout = 0.1  # Dropout rate

In [ ]:
# Initialize GPT-2 model and tokenizer
gpt2_model = GPT2LMHeadModel.from_pretrained('gpt2').to(device)
gpt2_model.eval()  # Freeze the GPT-2 model
gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Freeze all the parameters
for param in gpt2_model.parameters():
    param.requires_grad = False

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model):
        super(PositionalEncoding, self).__init__()
        self.d_model = d_model

    def forward(self, x):
        L, N = x.size(10), x.size(0)
        pos = torch.arange(L).unsqueeze(0).repest(N, 1).to(x.device)
        pos_embedding = self.calc_pos_embedding(pos)
        return x + pos_embedding
    def calc_pos_embedding(self, pos):
        pos = pos.float()
        factor = torch.exp(-torch.arange(0, self.d_model, 2).float() * (torch.log(torch.tensor(10000.0)) / self.d_model))
        sinusoid_inp = torch.ger(pos, factor)
        pos_embedding = torch.cat([sinusoid_inp.sin(), sinusoid_inp.cos()], dim=-1)
        return pos_embedding

In [ ]:
class TransformerDecoder(nn.Module):
    def __init__(self, d_model, num_heads, dim_feedforward, dropout, num_layers):
        super(TransformerDecoder, self).__init__()
        decoder_layer = nn.TransformerDecoderLayer(d_model, num_heads, dim_feedforward, dropout)
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers)
        self.output_layer = nn.Linear(d_model, output_dim)
    
    def forward(self, x, tgt):
        tgt = tgt.permute(1, 0, 2)
        output = self.transformer_decoder(tgt, x)
        output = self.output_layer(output)
        return output


In [ ]:
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.fc1(x)


In [ ]:
class MyModel(nn.Module):
    def __init__(self, d_model, num_heads, dim_feedforward, dropout, num_layers, output_dim):
        super(MyModel, self).__init__()
        self.positional_encoding = PositionalEncoding(d_model)
        self.decoder = TransformerDecoder(d_model, num_heads, dim_feedforward, dropout, num_layers)
        self.mlp = MLP(d_model, output_dim)
    
    def forward(self, x):
        x = self.positional_encoding(x)
        x = self.decoder(x)
        x = self.mlp(x)
        return x


In [ ]:
# Loss function
loss_fn = F.cross_entropy()

# Your training loop here
# where you calculate the loss as per your requirement and perform backpropagation

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    for i, (input_data, target) in enumerate(train_loader):
        input_data, target = input_data.to(device), target.to(device)

        # Forward pass through your model
        output = model(input_data)

        # Tokenize the output of your model
        tokens = gpt2_tokenizer(output.tolist(), return_tensors='pt').to(device)

        # Forward pass through GPT-2 model
        gpt2_output = gpt2_model(tokens).last_hidden_state

        # Remove the first 32 tokens from the GPT-2 output
        gpt2_output = gpt2_output[:, 32:]

        # Calculate loss
        loss = loss_fn(gpt2_output, target)

        # Backpropagate loss and update parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch: {epoch}, Loss: {loss.item()}')